In [ ]:
!pip install imutils

Imutils are a series of convenience functions to make basic image processing functions such as translation, rotation, resizing, skeletonization, and displaying Matplotlib images easier with OpenCV and both Python 2.7 and Python 3.

In [1]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import pickle
import cv2

In [2]:
import os
from glob import glob
import pandas as pd
import tensorflow as tf

In [3]:
print("[Info] loading dataset...")
data = []
labels = []
bboxes = []
imagePaths = []
i = 0


[Info] loading dataset...


In [4]:
#os.getcwd()

In [5]:
#base_path = os.path.sep.join([os.getcwd(), "Dataset"])

In [6]:
#base_path

In [7]:
base_path = "Dataset"

In [8]:
annotation_path = os.path.sep.join([base_path, "annotations"])

In [9]:
annotation_path

'Dataset\\annotations'

In [10]:
image_path = os.path.sep.join([base_path, "images"]) #pathseperator '\\'

In [11]:
image_path

'Dataset\\images'

In [12]:
#loop over all CSV files in the annotations folder
list = []
#list1 = []
for csvPath in paths.list_files(annotation_path, validExts= (".csv")):
    rows = open(csvPath).read().strip().split("\n")
   # list.append(rows)
    #row = csv.split("\n")
    #list.append(rows)
    
   
    
    for row in rows:
        row = row.strip().split(",")
        #list1.append(row)
        (filename, startX, startY, endX, endY, label) = row # assign variables
        
        imagePath = os.path.sep.join([image_path, label, filename]). replace(" ","")
        image =cv2.imread(imagePath)
        print(imagePath)
        if np.shape(image) != ():
            (h,w) = image.shape[:2] #because (rows, columns, channels) in image
             
            #scale the bounding box coordinates relative to the  spatial dimensions of the input image
            startX = float(startX)/w
            startY = float(startY)/h
            endX = float(endX)/w
            endY = float(endY)/h
            
            #load the image and preprocess it
            image = load_img(imagePath, target_size=(224,224)) #tf.keras.preprocessing.image.load_img already imported
            image = img_to_array(image)  #tf.keras.preprocessing.image.img_to_array
                               
            data.append(image)
            labels.append(label)
            bboxes.append((startX, startY, endX, endY))
            imagePaths.append(imagePath)                      

Dataset\images\airplane\image_0001.jpg
Dataset\images\airplane\image_0002.jpg
Dataset\images\airplane\image_0003.jpg
Dataset\images\airplane\image_0004.jpg
Dataset\images\airplane\image_0005.jpg
Dataset\images\airplane\image_0006.jpg
Dataset\images\airplane\image_0007.jpg
Dataset\images\airplane\image_0008.jpg
Dataset\images\airplane\image_0009.jpg
Dataset\images\airplane\image_0010.jpg
Dataset\images\airplane\image_0011.jpg
Dataset\images\airplane\image_0012.jpg
Dataset\images\airplane\image_0013.jpg
Dataset\images\airplane\image_0014.jpg
Dataset\images\airplane\image_0015.jpg
Dataset\images\airplane\image_0016.jpg
Dataset\images\airplane\image_0017.jpg
Dataset\images\airplane\image_0018.jpg
Dataset\images\airplane\image_0019.jpg
Dataset\images\airplane\image_0020.jpg
Dataset\images\airplane\image_0021.jpg
Dataset\images\airplane\image_0022.jpg
Dataset\images\airplane\image_0023.jpg
Dataset\images\airplane\image_0024.jpg
Dataset\images\airplane\image_0025.jpg
Dataset\images\airplane\i

In [13]:
data[1]

array([[[255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.],
        ...,
        [192., 177., 144.],
        [193., 179., 144.],
        [191., 183., 147.]],

       [[255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.],
        ...,
        [191., 176., 143.],
        [192., 178., 143.],
        [190., 182., 146.]],

       [[255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.],
        ...,
        [190., 175., 142.],
        [191., 177., 142.],
        [189., 179., 144.]],

       ...,

       [[255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.],
        ...,
        [181., 172., 141.],
        [183., 174., 145.],
        [180., 174., 142.]],

       [[255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.],
        ...,
        [180., 171., 140.],
        [181., 172., 143.],
        [178., 172., 140.]],

       [[255., 255., 255.],
        [255., 255., 255.],
        [255., 2

In [21]:
len(set(labels)) # we have 3 labels

3

In [22]:
 # covert the data in np arrays
data = np.array(data, dtype = "float32")/255.0
labels = np.array(labels) #string
bboxes = np.array(bboxes, dtype = "float32")
imagePaths = np.array(imagePaths) #string

In [23]:
# One Hot encoding
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [24]:
lb.classes_

array(['  motorcycle', ' airplane', ' face'], dtype='<U12')

In [25]:
len(lb.classes_)

3

In [26]:
labels

array([[0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       ...,
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0]])

In [27]:
#train test split
split = train_test_split(data, labels, bboxes, imagePaths, test_size= 0.2, random_state= 10)

In [30]:
split

[array([[[[0.3372549 , 0.32941177, 0.42352942],
          [0.2784314 , 0.27058825, 0.3647059 ],
          [0.2901961 , 0.28235295, 0.3764706 ],
          ...,
          [0.3647059 , 0.41960785, 0.3529412 ],
          [0.38431373, 0.43529412, 0.36862746],
          [0.42745098, 0.47058824, 0.40784314]],
 
         [[0.6313726 , 0.6392157 , 0.6901961 ],
          [0.59607846, 0.6039216 , 0.654902  ],
          [0.60784316, 0.6156863 , 0.6666667 ],
          ...,
          [0.49803922, 0.5529412 , 0.4862745 ],
          [0.5254902 , 0.5764706 , 0.50980395],
          [0.53333336, 0.5764706 , 0.5137255 ]],
 
         [[0.6627451 , 0.68235296, 0.69803923],
          [0.6392157 , 0.65882355, 0.6745098 ],
          [0.64705884, 0.6666667 , 0.68235296],
          ...,
          [0.52156866, 0.5764706 , 0.50980395],
          [0.50980395, 0.56078434, 0.49411765],
          [0.5372549 , 0.5803922 , 0.5176471 ]],
 
         ...,
 
         [[0.5764706 , 0.6039216 , 0.57254905],
          [0.54509

In [31]:
#assign the data labels
(trainImages, testImages) = split[:2]
(trainLabels, testLabels) = split[2:4]
(trainBBoxes, testBBoxes) =split[4:6]
(trainPaths, testPaths) = split[6:]


In [33]:
#Load the VGG16 Network, ensuring the head FC layers are left off

vgg = VGG16(weights = 'imagenet', include_top = False, input_tensor= Input(shape=(224,224,3)))

58892288/58889256 [==============================] - 18s 0us/step


In [34]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [35]:
# freeze all VGG layers so they will *not* be updated during the training process
vgg.trainable = False

In [41]:
# flatten the max-pooling output of VGG
flatten = vgg.output
flatten = Flatten()(flatten) #layer itself 

In [48]:
# A FC layer header to output the predicted bounding box coordinates
bboxHead = Dense(128, activation = 'relu')(flatten)
bboxHead = Dense(64, activation = 'relu')(bboxHead)
bboxHead = Dense(32, activation = 'relu')(bboxHead)
bboxHead = Dense(4, activation = 'sigmoid', name ='bounding_box')(bboxHead)

In [43]:
# Construct a parallel second FC layer head to predict the class label
labelHead = Dense(512, activation = 'relu')(flatten)
labelHead = Dropout(0.5)(labelHead)
labelHead = Dense(512, activation = 'relu')(labelHead)
labelHead = Dropout(0.5)(labelHead)
labelHead = Dense(len(lb.classes_), activation = 'softmax', name = 'class_label')(labelHead)

In [49]:
#Put together our model which accepts input images with two outputs one for Bboxes and another for class labels
#not model = sequential()
model = Model(inputs = vgg.input, outputs = (bboxHead, labelHead))

In [45]:
losses = {
    'class_label' : 'categorical_crossentropy',
    'bounding_box' : 'mean_squared_error',
}

In [46]:
lossWeights ={
    'class_label' : 1.0,
    'bounding_box' : 1.0
}

In [50]:
opt = Adam(lr = 1e-4)
model.compile(loss = losses, optimizer = opt, metrics = ['accuracy'], loss_weights = lossWeights)
print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 224, 224, 64) 1792        input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 224, 224, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 112, 112, 64) 0           block1_conv2[0][0]               
____________________________________________________________________________________________

In [51]:
#construct a dictionary for our target training outputs
trainTargets = {
    'class_label' : trainLabels, #from previous
    'bounding_box' : trainBBoxes
}

In [53]:
#construct a second dictionary for our target testing outputs
testTargets = {
    'class_label' : testLabels,
    'bounding_box' : testBBoxes
}

In [54]:
#train the network for Bounding box regression and class label prediction
print("[INFO] trainig model.....")
History = model.fit(trainImages, trainTargets, validation_data =(testImages, testTargets), batch_size = 32,epochs = 10, verbose = 1)


[INFO] trainig model.....
Epoch 1/10
51/51 [==============================] - 854s 17s/step - loss: 0.5000 - bounding_box_loss: 0.0275 - class_label_loss: 0.4724 - bounding_box_accuracy: 0.6426 - class_label_accuracy: 0.8018 - val_loss: 0.0041 - val_bounding_box_loss: 0.0014 - val_class_label_loss: 0.0027 - val_bounding_box_accuracy: 0.8771 - val_class_label_accuracy: 1.0000
Epoch 2/10
51/51 [==============================] - 828s 16s/step - loss: 0.0181 - bounding_box_loss: 0.0013 - class_label_loss: 0.0168 - bounding_box_accuracy: 0.8512 - class_label_accuracy: 0.9970 - val_loss: 0.0041 - val_bounding_box_loss: 9.8933e-04 - val_class_label_loss: 0.0031 - val_bounding_box_accuracy: 0.8550 - val_class_label_accuracy: 1.0000
Epoch 3/10
51/51 [==============================] - 2440s 49s/step - loss: 0.0140 - bounding_box_loss: 8.5900e-04 - class_label_loss: 0.0131 - bounding_box_accuracy: 0.8565 - class_label_accuracy: 0.9973 - val_loss: 0.0013 - val_bounding_box_loss: 8.6690e-04 - val_c

In [55]:
 model.save('bbox model', save_format  = 'H5')      

INFO:tensorflow:Assets written to: bbox model\assets
